# Joining Tables with `dfply`

In [2]:
import pandas as pd
from dfply import *

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [1]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [3]:
dept = pd.read_csv("./data/department.csv")
dept

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [4]:
empl = pd.read_csv("./data/employee.csv")
empl

,LastName,DeptID
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,NaN


## Left Outer Join

Use `left_table >> dfply.left_join(right_table, by=common_col_name)`

In [8]:
empl >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN


## Rename when the column names are different

In [9]:
empl2 = empl.fillna(0) >> rename(deptIDL = 'DeptID') 
empl2

,LastName,deptIDL
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,0.0


In [10]:
empl2 >> left_join(dept, by='DeptID')

KeyError: 'DeptID'

In [11]:
empl2 >> rename(DeptID = 'deptIDL') >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,0.0,NaN


## Right Outer Join

Use `right_table >> dfply.right_join(left_table, by=common_col_name)`

In [12]:
empl >> right_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,NaN,35.0,Marketing


## Full Outer Join

Use `left_table >> dfply.outer_join(right_table, by=common_col_name)`

In [13]:
empl >> outer_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN
6,NaN,35.0,Marketing


## Inner Join

Use `left_table >> dfply.inner_join(right_table, by=common_col_name)`

In [14]:
empl >> inner_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical


## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [15]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/Teams.csv")

In [16]:

batting, teams = [pd.read_csv(f) for f in files]

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [17]:
batting.head(2)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0


#### Selecting down to the necessary columns

In [18]:
batting_select = (batting
                  >> select(X.teamID,
                            X.yearID,
                            X.R)
                 )
batting_select.head()

,teamID,yearID,R
0,TRO,1871,0
1,RC1,1871,30
2,CL1,1871,28
3,WS3,1871,28
4,RC1,1871,29


In [19]:
batting_select.shape

(107429, 3)

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [27]:
teams.head(2)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1


#### Selecting down to the necessary columns

In [20]:
teams_select = (teams >> select(X.yearID,
                                X.teamID,
                                X.name,
                                X.park))
teams_select.head()

,yearID,teamID,name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


In [21]:
teams_select.shape

(2925, 4)

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `by`

In [22]:
df_joined = batting_select >> left_join(teams_select, by=['teamID', 'yearID'])
df_joined.head()

,teamID,yearID,R,name,park
0,TRO,1871,0,Troy Haymakers,Haymakers' Grounds
1,RC1,1871,30,Rockford Forest Citys,Agricultural Society Fair Grounds
2,CL1,1871,28,Cleveland Forest Citys,National Association Grounds
3,WS3,1871,28,Washington Olympics,Olympics Grounds
4,RC1,1871,29,Rockford Forest Citys,Agricultural Society Fair Grounds


#### Filter, group by, and aggregate to get the desired result.

In [23]:
runs_per_team09 = (df_joined
                   >> filter_by(X.yearID == 2009)
                   >> group_by(X.name, X.park)
                   >> summarise(total_runs = X.R.sum()))
runs_per_team09.head()

,park,name,total_runs
0,Chase Field,Arizona Diamondbacks,720
1,Turner Field,Atlanta Braves,735
2,Oriole Park at Camden Yards,Baltimore Orioles,741
3,Fenway Park II,Boston Red Sox,872
4,Wrigley Field,Chicago Cubs,707


In [24]:
runs_per_team09.shape

(30, 3)

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example 

In [25]:
len(batting_select)*len(teams_select)

314229825

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [26]:
runs = (batting_select
        >> filter_by(X.yearID == 2009)
        >> group_by(X.teamID)
        >> summarise(total_runs = X.R.sum()))

runs.head()

,teamID,total_runs
0,ARI,720
1,ATL,735
2,BAL,741
3,BOS,872
4,CHA,724


#### Filter down to 2009 teams

In [27]:
teams09 = (teams_select
           >> filter_by(X.yearID == 2009)
           >> drop(X.yearID)
          )
teams09.head()

,teamID,name,park
2595,ARI,Arizona Diamondbacks,Chase Field
2596,ATL,Atlanta Braves,Turner Field
2597,BAL,Baltimore Orioles,Oriole Park at Camden Yards
2598,BOS,Boston Red Sox,Fenway Park II
2599,CHA,Chicago White Sox,U.S. Cellular Field


In [36]:
len(runs)*len(teams09)

900

In [28]:
runs_per_team09_new = (runs 
                       >> left_join(teams09, by='teamID')
                       >> drop(X.teamID))
runs_per_team09_new.head()

,total_runs,name,park
0,720,Arizona Diamondbacks,Chase Field
1,735,Atlanta Braves,Turner Field
2,741,Baltimore Orioles,Oriole Park at Camden Yards
3,872,Boston Red Sox,Fenway Park II
4,724,Chicago White Sox,U.S. Cellular Field


In [38]:
runs_per_team09_new.shape

(30, 3)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 1 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join methods presented above.

In [30]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/People.csv",
         "./data/baseball/core/Teams.csv")

In [ ]:
#I am 95% certain that no one has ever hit 100 homeruns in a single season...

In [31]:
# Your code here

batting, people, teams = [pd.read_csv(f) for f in files]

In [32]:
batting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107429 entries, 0 to 107428
Data columns (total 22 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   playerID  107429 non-null  object 
 1   yearID    107429 non-null  int64  
 2   stint     107429 non-null  int64  
 3   teamID    107429 non-null  object 
 4   lgID      106691 non-null  object 
 5   G         107429 non-null  int64  
 6   AB        107429 non-null  int64  
 7   R         107429 non-null  int64  
 8   H         107429 non-null  int64  
 9   2B        107429 non-null  int64  
 10  3B        107429 non-null  int64  
 11  HR        107429 non-null  int64  
 12  RBI       106673 non-null  float64
 13  SB        105061 non-null  float64
 14  CS        83888 non-null   float64
 15  BB        107429 non-null  int64  
 16  SO        105329 non-null  float64
 17  IBB       70778 non-null   float64
 18  HBP       104612 non-null  float64
 19  SH        101360 non-null  float64
 20  SF  

In [35]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [33]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20090 entries, 0 to 20089
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   playerID      20090 non-null  object 
 1   birthYear     19976 non-null  float64
 2   birthMonth    19808 non-null  float64
 3   birthDay      19666 non-null  float64
 4   birthCountry  20029 non-null  object 
 5   birthState    19555 non-null  object 
 6   birthCity     19916 non-null  object 
 7   deathYear     9858 non-null   float64
 8   deathMonth    9857 non-null   float64
 9   deathDay      9856 non-null   float64
 10  deathCountry  9855 non-null   object 
 11  deathState    9805 non-null   object 
 12  deathCity     9850 non-null   object 
 13  nameFirst     20053 non-null  object 
 14  nameLast      20090 non-null  object 
 15  nameGiven     20053 non-null  object 
 16  weight        19273 non-null  float64
 17  height        19353 non-null  float64
 18  bats          18910 non-nu

In [34]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Data columns (total 48 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   yearID          2925 non-null   int64  
 1   lgID            2875 non-null   object 
 2   teamID          2925 non-null   object 
 3   franchID        2925 non-null   object 
 4   divID           1408 non-null   object 
 5   Rank            2925 non-null   int64  
 6   G               2925 non-null   int64  
 7   Ghome           2526 non-null   float64
 8   W               2925 non-null   int64  
 9   L               2925 non-null   int64  
 10  DivWin          1380 non-null   object 
 11  WCWin           744 non-null    object 
 12  LgWin           2897 non-null   object 
 13  WSWin           2568 non-null   object 
 14  R               2925 non-null   int64  
 15  AB              2925 non-null   int64  
 16  H               2925 non-null   int64  
 17  2B              2925 non-null   i

In [39]:
#first, I need every player that hit at least 101 home runs in at least one year

batting_100HR = (batting
                >> filter_by(X["HR"] > 100)


)

batting_100HR

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP


In [44]:
#well, this is awkward, I can still get the columns at least assuming there was anybody who fit this

#I would need the columns playerID and TeamID for joins

batting_100HR = (batting
                >> filter_by(X["HR"] > 100)
                >> select(X["playerID"], X["teamID"])
)

batting_100HR


,playerID,teamID


In [50]:
#now, I would need to join this onto players to get the player name

#I'll use a left join

batting_100HR = (batting
                >> filter_by(X["HR"] > 100)
                >> select(X["playerID"], X["teamID"])
                >> left_join(people, by="playerID")
                >> select(X["nameFirst"], X["nameLast"], X["nameGiven"], X["teamID"])
)

batting_100HR.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nameFirst  0 non-null      object
 1   nameLast   0 non-null      object
 2   nameGiven  0 non-null      object
 3   teamID     0 non-null      object
dtypes: object(4)
memory usage: 0.0+ bytes


In [55]:
#now to grab the teams

batting_100HR_players = (batting
                >> filter_by(X["HR"] > 100)
                >> select(X["playerID"], X["teamID"])
                >> left_join(people, by="playerID")
                >> select(X["nameFirst"], X["nameLast"], X["nameGiven"], X["teamID"])
                >> left_join(teams, by="teamID")
                >> select(X["nameFirst"], X["nameLast"], X["nameGiven"], X["teamID"], X["name"])
)

batting_100HR_players


,nameFirst,nameLast,nameGiven,teamID,name


But this will grab multiple teams, so we I will adjust it slightly

In [91]:
batting_100HR_players = (batting
                >> filter_by(X["HR"] > 60)
                >> select(X["playerID"], X["teamID"], X["yearID"])
                >> left_join(teams, by=["teamID", "yearID"])
                >> select(X["playerID"], X["teamID"], X["yearID"], X["name"])
                >> left_join(people, by="playerID")
                >> select(X["nameFirst"], X["nameLast"], X["nameGiven"], X["name"], X["yearID"])
)

batting_100HR_players

,nameFirst,nameLast,nameGiven,name,yearID
0,Roger,Maris,Roger Eugene,New York Yankees,1961
1,Mark,McGwire,Mark David,St. Louis Cardinals,1998
2,Sammy,Sosa,Samuel Peralta,Chicago Cubs,1998
3,Mark,McGwire,Mark David,St. Louis Cardinals,1999
4,Sammy,Sosa,Samuel Peralta,Chicago Cubs,1999
5,Barry,Bonds,Barry Lamar,San Francisco Giants,2001
6,Sammy,Sosa,Samuel Peralta,Chicago Cubs,2001


As funny as this is, I'll run it again with more than 60 to show that it works

In [90]:
batting_60HR_players = (batting
                >> filter_by(X["HR"] > 60)
                >> select(X["playerID"], X["teamID"], X["yearID"])
                >> left_join(teams, by=["teamID", "yearID"])
                >> select(X["playerID"], X["teamID"], X["yearID"], X["name"])
                >> left_join(people, by="playerID")
                >> select(X["nameFirst"], X["nameLast"], X["nameGiven"], X["name"], X["yearID"])
)

batting_60HR_players

,nameFirst,nameLast,nameGiven,name,yearID
0,Roger,Maris,Roger Eugene,New York Yankees,1961
1,Mark,McGwire,Mark David,St. Louis Cardinals,1998
2,Sammy,Sosa,Samuel Peralta,Chicago Cubs,1998
3,Mark,McGwire,Mark David,St. Louis Cardinals,1999
4,Sammy,Sosa,Samuel Peralta,Chicago Cubs,1999
5,Barry,Bonds,Barry Lamar,San Francisco Giants,2001
6,Sammy,Sosa,Samuel Peralta,Chicago Cubs,2001
